In [ ]:
import requests
import dateparser
import pandas as pd
from sqlalchemy import create_engine

API_KEY = "bf5c92db4ce2ac90addf545eed810f1a93d5241c3b6ca5bca9a0603f2625d5af"

def get_news_serpapi(query):
    url = "https://serpapi.com/search.json"
    params = {
        "q": query,
        "tbm": "nws",
        "hl": "id",
        "gl": "id",
        "api_key": API_KEY
    }

    res = requests.get(url, params=params)
    data = res.json()

    if "news_results" not in data:
        print(f"Tidak ada hasil ditemukan untuk: {query}")
        return pd.DataFrame()

    articles = data["news_results"]
    df = pd.DataFrame([{
        "title": a["title"],
        "source": a.get("source"),
        "date": a.get("date"),
        "link": a.get("link"),
        "snippet": a.get("snippet"),
        "query": query  # tambahkan kolom asal query
    } for a in articles])

    return df

# Daftar kata kunci
keywords = ["Bank Muamalat Indonesia", "Muamalat Digital", "Bank Muamalat","BPKH"]

# Gabungkan hasil dari semua kata kunci
all_results = []

for kw in keywords:
    print(f"Mencari berita untuk: {kw}")
    df = get_news_serpapi(kw)
    if not df.empty:
        all_results.append(df)

# Gabungkan semua hasil jadi satu DataFrame
df_all = pd.concat(all_results, ignore_index=True)
# Asumsikan df_all adalah DataFrame hasil scraping berita
def parse_date(date_string):
    # Gunakan settings untuk timezone Jakarta dan prefer past
    return dateparser.parse(date_string, settings={'PREFER_DATES_FROM': 'past', 'TIMEZONE': 'Asia/Jakarta'})

# Terapkan ke kolom 'date'
df_all['parsed_date'] = df_all['date'].apply(parse_date)

# Ubah ke timezone-aware datetime (optional)
df_all['parsed_date'] = pd.to_datetime(df_all['parsed_date'])

# # Simpan hasil ke CSV jika perlu
# df_all.to_csv("berita_bank_muamalat.csv", index=False, encoding="utf-8-sig")

# Simpan ke CSV
# df_all.to_csv("berita_bank_muamalat.csv", index=False, encoding="utf-8-sig")

# df_all
# # Koneksi ke PostgreSQL lokal
# dbname = "........."
# user = "........."
# password = "Mu@......."
# host = "localhost"
# port = "........"  # 

# # Buat koneksi
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}")

# # Simpan ke tabel (misalnya nama tabel: `berita_muamalat`)
# df_all.to_sql("berita_muamalat", con=engine, if_exists="replace", index=False)

# print("Berhasil dimasukkan ke PostgreSQL.")

Mencari berita untuk: Bank Muamalat Indonesia
Mencari berita untuk: Muamalat Digital
Mencari berita untuk: Bank Muamalat
Mencari berita untuk: BPKH


In [ ]:
#Simpan ke CSV
# df_all.to_csv("berita_bank_muamalat.csv", index=False, encoding="utf-8-sig")

In [ ]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

def insert_df_to_postgres(df, table_name, user, password, host, port, dbname):
    if df.empty:
        print("[INFO] DataFrame kosong, tidak ada yang disimpan.")
        return

    # Hanya kolom yang sesuai dengan struktur tabel PostgreSQL
    allowed_columns = ["title", "source", "date", "link", "snippet", "query", "parsed_date"]
    df = df[[col for col in allowed_columns if col in df.columns]]

    encoded_password = quote_plus(password)
    engine_url = f"postgresql+psycopg2://{user}:{encoded_password}@{host}:{port}/{dbname}"
    engine = create_engine(engine_url)

    try:
        df.to_sql(
            table_name,
            con=engine,
            if_exists="append",  # atau "replace" jika kamu ingin overwrite
            index=False,
            method="multi",
            chunksize=500
        )
        print(f"[OK] Data berhasil disimpan ke tabel '{table_name}'.")
    except Exception as e:
        print(f"[ERROR] Gagal menyimpan ke database: {e}")

insert_df_to_postgres(df_all, "muamalat_news", "..........", "Mu@........", "......", ".....", "........")

[OK] Data berhasil disimpan ke tabel 'muamalat_news'.


In [10]:
df_all

,title,source,date,link,snippet,query,parsed_date,ingest_time
0,Bank Muamalat Catatkan Volume Transaksi QRIS R...,SWA.co.id,1 hari lalu,https://swa.co.id/read/460501/bank-muamalat-ca...,PT Bank Muamalat Indonesia Tbk atau Muamalat D...,Bank Muamalat Indonesia,2025-06-11 11:09:50.822143,2025-06-12 11:15:23.348120
1,Transaksi QRIS di Muamalat DIN Tumbuh Double D...,Kr Jogja,19 jam yang lalu,https://www.krjogja.com/keuangan/1246127998/tr...,Layanan QRIS di mobile banking PT Bank Muamala...,Bank Muamalat Indonesia,2025-06-11 16:09:51.110246,2025-06-12 11:15:23.348120
2,Bank Muamalat catat transaksi QRIS tumbuh dua ...,ANTARA News,1 hari lalu,https://www.antaranews.com/berita/4888401/bank...,PT Bank Muamalat Indonesia Tbk mencatat transa...,Bank Muamalat Indonesia,2025-06-11 11:09:51.118636,2025-06-12 11:15:23.348120
3,"Gagal Tes OJK, Bankir Berprestasi Ini Gagal Ja...",kontan.co.id,2 minggu lalu,https://keuangan.kontan.co.id/news/gagal-tes-o...,Kukuh diangkat sebagai direktur Bank Muamalat ...,Bank Muamalat Indonesia,2025-05-29 11:09:51.123622,2025-06-12 11:15:23.348120
4,Bank Muamalat catat transaksi QRIS tumbuh dua ...,ANTARA News,1 hari lalu,https://www.antaranews.com/berita/4888401/bank...,PT Bank Muamalat Indonesia Tbk mencatat transa...,Muamalat Digital,2025-06-11 11:09:51.129626,2025-06-12 11:15:23.348120
5,Transaksi QRIS di Muamalat DIN Tumbuh Dua Digit,SWA.co.id,1 hari lalu,https://swa.co.id/read/460488/transaksi-qris-d...,Layanan QRIS dan mobile banking di Muamalat DI...,Muamalat Digital,2025-06-11 11:09:51.138607,2025-06-12 11:15:23.348120
6,"Bank Muamalat luncurkan ATLAS, bank digital Is...",Majalah ICT,1 minggu lalu,https://www.majalahict.com/bank-muamalat-luncu...,"MAJALAH ICT – Jakarta. Backbase, pemimpin glob...",Muamalat Digital,2025-06-05 11:09:51.143594,2025-06-12 11:15:23.348120
7,Bank Muamalat luncurkan bank digital islami pe...,Tech in Asia Indonesia,5 hari lalu,https://id.techinasia.com/berita/bank-muamalat...,Backbase mengumumkan peluncuran ATLAS oleh Ban...,Muamalat Digital,2025-06-07 11:09:51.152784,2025-06-12 11:15:23.348120
8,"Bank Muamalat launches ATLAS, Malaysia's first...",riauone.com,5 hari lalu,https://www.riauone.com/global/Bank-Muamalat-l...,"KUALA LUMPUR, MALAYSIA - 5 June 2025 - Backbas...",Muamalat Digital,2025-06-07 11:09:51.161219,2025-06-12 11:15:23.348120
9,Bank Muamalat Catatkan Volume Transaksi QRIS R...,SWA.co.id,1 hari lalu,https://swa.co.id/read/460501/bank-muamalat-ca...,PT Bank Muamalat Indonesia Tbk atau Muamalat D...,Muamalat Digital,2025-06-11 11:09:51.168702,2025-06-12 11:15:23.348120
